# Caribbean Karst Habitat

This is code for Southeast Conservation Blueprint 2023. Written by Rua Mordecai.

## Import libraries and define variables

In [65]:
import os
import arcpy

In [66]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
landfire = r"D:\Landcover\LF2020_Puerto_Rico_Virgin_Islands_220_IA\LF2020_PRVI_220_IA\LF2020_EVT_220_PRVI\Tif\LV20_EVT_220.tif"
PRKarst = r"D:\Blueprint\2023\Caribbean\Indicators\Karst\PR_DRNA\karst\karst\karstpublichearingSept2013\APE-RC.shp"
PRKarstBuffer = r"D:\Blueprint\2023\Caribbean\Indicators\Karst\PR_DRNA\karst\karst\karstpublichearingSept2013\Buffer50M.shp"
USGSVIKarst = r"D:\Blueprint\2023\Caribbean\Indicators\Karst\USGS_USKarstMap\Shapefiles\PuertoRico_VirginIslands\VIcarbonates.shp"
NPSGeology = r"D:\Blueprint\2023\Caribbean\Indicators\Karst\NPS\buisshp\buisglg.shp"

In [67]:
# define final output name
IndicatorFileName = "CaribbeanKarstHabitat.tif"

In [68]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [69]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

### Prep data

In [70]:
# Switch to working directory
arcpy.env.workspace = OutputWorkspace

In [36]:
# Make a version of USGS Karst without St. Croix. 
# Those data are very coarse and we have better info for St. Croix
out_raster = arcpy.management.SelectLayerByAttribute(USGSVIKarst, "NEW_SELECTION", "UNIT_NAME <> 'Kingshill Marl'", None)

In [37]:
# Export the shapefile with St. Croix removed
out_raster2 = arcpy.conversion.FeatureClassToFeatureClass(out_raster, OutputWorkspace, "USGSVIKarstNoStCroix.shp")

In [38]:
# Extract karst from NPS St. Croix Geology. MIkh is the symbol for Kingshill Marl.
out_raster = arcpy.management.SelectLayerByAttribute(NPSGeology, "NEW_SELECTION", "GLG_SYM = 'MIkh'", None)

In [39]:
# Export the shapefile with St. Croix karst
out_raster2 = arcpy.conversion.FeatureClassToFeatureClass(out_raster, OutputWorkspace, "StCroixKarst.shp")

### Begin analysis

In [40]:
# combine karst from Puerto Rico and Virgin Islands
arcpy.management.Merge(["USGSVIKarstNoStCroix.shp", "StCroixKarst.shp", PRKarst, PRKarstBuffer], "CaribbeanKarst.shp")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Karst\\working\\CaribbeanKarst.shp'>

In [41]:
# add a field to convert to raster, give a value of 1
arcpy.management.CalculateField("CaribbeanKarst.shp", "raster", "1", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Karst\\working\\CaribbeanKarst.shp'>

In [42]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    arcpy.conversion.FeatureToRaster(in_features="CaribbeanKarst.shp", field="raster", out_raster="CaribbeanKarstRaster.tif", cell_size=CaribbeanRaster)

In [43]:
# Make no data zero
out_raster = arcpy.sa.Reclassify("CaribbeanKarstRaster.tif", "Value", "NODATA 0;1 1", "DATA"); out_raster.save("CaribbeanKarstRasterZ.tif")

In [44]:
# Use the same alteration categories as the landscape condition indicator
# Reclassify Landfire based on amount of alteration. 3 is natural, 2 is altered, and 1 is heavily altered. 
# Medium and High intensity developed are 1, anything else categorized as "Developed","Agriculture", or "Barren" in "EVT_LF" 
# or "Agriculture - Pasture and Hay" or "Bush fruit and berries" in "EVT_NAME" is 2, everything else is 3.
#"Open Water" is NoDATA
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    out_raster = arcpy.sa.Reclassify(landfire, "Value", "-9999 NODATA;7292 NODATA;7295 7296 2;7297 7298 1;7299 7300 2;7754 2;7755 2;7838 2;7861 7888 3;9303 9355 3;9803 3", "DATA"); out_raster.save("LandscapeConditionBins.tif")

In [45]:
# Combine rasters for indicator categories
out_raster = arcpy.sa.Times("CaribbeanKarstRasterZ.tif", "LandscapeConditionBins.tif"); out_raster.save("indicatorBins.tif")

In [46]:
# clip raster to Caribbean extent
out_raster = arcpy.sa.ExtractByMask("indicatorBins.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [71]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Not identified as karst'    
    elif value == 1:
        return '1 = Karst with developed landcover'
    elif value == 2:
        return '2 = Karst with pasture, crops, or open space in developed area' 
    elif value == 3:
        return '3 = Karst with natural landcover'             
"""

In [72]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanKarstHabitat.tif'>

In [73]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 3:
		return 199
	if Value == 2:
		return 230
	if Value == 1:
		return 252
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 3:
		return 0
	if Value == 2:
		return 108
	if Value == 1:
		return 192
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 3:
		return 99
	if Value == 2:
		return 153
	if Value == 1:
		return 238
	if Value == 0:
		return 255
	else:
		return 255
		
"""

In [74]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanKarstHabitat.tif'>